In [4]:
import pandas as pd
import numpy as np
from pytrends.request import TrendReq
import pytrends as pt
import json

%matplotlib notebook

In [5]:
#Search domains codes for Google trends.
search_domains = {
    'Fashion & Style': 185,
    'Fashion Designers & Collections': 98,
    'Fashion Modeling': 1155,
    'Shopping': 18,
    'Antiques & Collectibles': 64,
    'Apparel': 68,
    'Apparel Services': 1228,
    'Athletic Apparel': 983,
    'Casual Apparel': 984,
    'T-Shirts': 428,
    "Children's Clothing": 985,
    'Clothing Accessories': 124,
    'Gems & Jewelry': 350,
    'Handbags & Purses': 986,
    'Watches': 987,
    'Costumes': 988,
    'Eyewear': 989,
    'Eyeglasses & Contacts': 1224,
    'Footwear': 697,
    'Formal Wear': 990,
    'Headwear': 991,
    "Men's Clothing": 992,
    'Outerwear': 993,
    'Sleepwear': 994,
    'Swimwear': 995,
    'Undergarments': 530,
    'Uniforms & Workwear': 996,
    "Women's Clothing": 997
}

In [57]:
#Reading the Attribute file.
atr_file = 'length.json'
df = pd.read_json(atr_file)
df

,Length
0,Ankle length
1,3/4 length
2,Midi
3,Short
4,Long
5,Mini
6,Maxi


In [58]:
#Getting a Pandas Series instance of attribute fields.
cat = df[df.columns[0]]
cat

0    Ankle length
1      3/4 length
2            Midi
3           Short
4            Long
5            Mini
6            Maxi
Name: Length, dtype: object

In [54]:
#####This functions returns a list of 4 attributes which have the maximum trend score on google trends when queried 5 at a time.#####

def max4(cat):
    att = [0,1,2,3,4]
    tr_lst = []
    if(len(cat) < 5):
        return tr_scr(cat,range(0,len(cat)))
    elif(len(cat) == 5):
        i_stack = []
        tr_lst = tr_scr(cat,att)
    else:
        i_stack = list(range(5,len(cat)))
    while(len(i_stack) > 0):
        tr_lst = tr_scr(cat,att)
        min_i = np.argmin(tr_lst)
        att.remove(att[min_i])
        att.append(i_stack.pop())
    
    min_i = np.argmin(tr_lst)
    att.remove(att[min_i])
    return att
        
    
######################## Function to get the mean trend score for attributes of a category #####################################    

def tr_scr(cat,att):
    trend = TrendReq(hl='en-US', tz=360)
    #field = [cat[att[0],cat[att[1]],cat[att[2]],cat[att[3]],cat[att[4]]]
    field = [cat[i] for i in att]
    kw_list = field
    trend.build_payload(kw_list, cat=search_domains['Apparel'], timeframe='today 3-m', geo='', gprop='')
    df = trend.interest_over_time()
    return list(df.mean())[:-1]

####################Funtion that returns the trend scores for last 3 months of the non max attributes of the category.############################

def tr_scr_nmx(cat,max_4):
    fab_scr = []
    max4_ = set(max_4)
    all_c = set(range(0,len(cat)))
    nmx = all_c - max4_
    nmx #Set of Non Max attribute indices.
    for i in nmx:
        trend = TrendReq(hl='en-US', tz=360)
        field = [cat[max_4[0]],cat[max_4[1]],cat[max_4[2]],cat[max_4[3]],cat[i]]
        kw_list = field
        trend.build_payload(kw_list, cat=search_domains['Apparel'], timeframe='today 3-m', geo='', gprop='')
        df = trend.interest_over_time()
        fab_scr.append(list(df.mean())[:-1])
    return fab_scr


################################## Making the trend score Dictionary ##########################################################


def mk_dic(scr,ct,m4):
    
    max4_ = set(max_4)
    all_c = set(range(0,len(cat)))
    nx = all_c - max4_
    scr_dic = {}
    i = 0
    for d in scr:  
        scr_dic[ct[list(nx)[i]]] = d[-1]
        i+=1
    j = 0
    a = np.zeros(5)
    for d in scr:
        a+=np.array(d)
        j+=1    
    a/=j
    for i in range(0,4):
        scr_dic[ct[m4[i]]] = a[i]
        
    return scr_dic



In [59]:
max_4 = max4(cat)#Using max_4 function
max_4

In [61]:
scr = tr_scr_nmx(cat,max_4) #Calculating relative trend score for the Non Max attributes and storing it style_scr.
scr 

[[63.285714285714285,
  84.96703296703296,
  28.197802197802197,
  36.417582417582416,
  0.054945054945054944],
 [63.285714285714285,
  84.96703296703296,
  28.197802197802197,
  36.417582417582416,
  0.23076923076923078],
 [63.285714285714285,
  84.96703296703296,
  28.197802197802197,
  36.417582417582416,
  16.21978021978022]]

In [62]:
scr_dic = mk_dic(scr,cat,max_4)
scr_dic

{'Ankle length': 0.054945054945054944,
 '3/4 length': 0.23076923076923078,
 'Midi': 16.21978021978022,
 'Short': 63.285714285714285,
 'Long': 84.96703296703296,
 'Maxi': 28.1978021978022,
 'Mini': 36.417582417582416}

In [35]:
pd.Series(scr_dic).to_json(attr_file + '_score.json') #To save the Dictionary into a json file.

In [42]:
scores = [
    'category_score.json',
    'collar_score.json',
    'Fabric_scr.json',
    'fabric_work.json',
    'leg_score.json',
    'length_score.json',
    'neck_score.json',
    'pattern_score.json',
    'sleeve_length_score.json',
    'sleeve_style_score.json',
    'style_score.json'
]

In [44]:
scores_ = {}
for scr in scores:
    f = open(scr)
    d = json.load(f)
    scores_.update(d)
    
scores_    

{'Boilersuits': 0.043956044,
 'Playsuits': 2.6483516484,
 'Rompers': 14.3516483516,
 'Dungarees': 12.4945054945,
 'Overalls': 27.9010989011,
 'Bodysuit': 31.0989010989,
 'Jumpsuit': 73.5494505495,
 'Notch collar': 1.6333333333,
 'Shawl collar': 5.5,
 'Revere collar': 0.9222222222,
 'Point collar': 1.2333333333,
 'Sailor collar': 0.9333333333,
 'Flat collar': 1.0888888889,
 'Bertha collar': 0.1,
 'Jabot collar': 0.1555555556,
 'Detachable collar': 0.8111111111,
 'Rolled collar': 0.3444444444,
 'Turn down collar': 0.8444444444,
 'Spread collar': 2.7666666667,
 'Mandarin collar': 4.9888888889,
 'Turtleneck': 69.3444444444,
 'Frill collar': 1.4333333333,
 'Beaded': 5.5168539326,
 'Chambray': 2.0224719101,
 'Charmeuse': 0.0674157303,
 'Chiffon': 6.8651685393,
 'Cotton Blend': 0.3258426966,
 'Jersey': 30.3146067416,
 'Knit': 16.7640449438,
 'Linen': 21.2359550562,
 'Linen Blend': 0.1123595506,
 'Mesh': 26.4382022472,
 'Satin': 23.191011236,
 'Silk Blend': 0.0112359551,
 'Spandex': 6.20224719

In [45]:
pd.Series(scores_).to_json('ALL_scores.json')